### Multi-Step Chained Reasoning

Setup the LLM

In [9]:

import os
import json
from openai import OpenAI
from pydantic import BaseModel, Field, ValidationError

# Initialize the OpenAI Client
# Ensure you have your API key ready
client = OpenAI()

### Defining the "Action" Layer (Structured Tools)

In this cell, we define the Structured Tools. We use Pydantic to ensure that even if the LLM makes a mistake, the system validates the data before execution.

In [13]:
# Pydantic validates and structures tool outputs
class InventoryCheck(BaseModel):
    item: str = Field(..., description="The name of the item to check")


class RestockRequest(BaseModel):
    item: str = Field(..., description="The name of the item to restock")
    quantity: int = Field(..., ge=1, le=100, description="Quantity to order (1-100)")


# Humans remain in control of execution boundaries
class MockCompanySystem:
    def __init__(self):
        # Initial internal state
        self.inventory = {"keyboard": 0, "laptop": 12, "monitor": 5}
        self.audit_log = []

    def check_stock(self, item_data: InventoryCheck):
        item = item_data.item.lower()
        count = self.inventory.get(item, 0)
        status = f"Current stock for {item}: {count}"
        self.audit_log.append(f"LOG: Checked {item}. Result: {count}")
        return status

    def restock_item(self, restock_data: RestockRequest):
        item = restock_data.item.lower()
        self.inventory[item] = self.inventory.get(item, 0) + restock_data.quantity
        status = f"Successfully ordered {restock_data.quantity} units of {item}."
        self.audit_log.append(f"LOG: Restocked {item} by {restock_data.quantity}")
        return status


# Initialize the system instance
backend_system = MockCompanySystem()

### Defining Tool Schemas for the LLM
Here, we define the "Toolkit" that is passed to the AI so it knows its boundaries.

In [14]:
# Toolkits organize tools by domain to reduce cognitive load 
tools = [
    {
        "type": "function",
        "function": {
            "name": "check_stock",
            "description": "Check current inventory levels for a specific item.",
            "parameters": {
                "type": "object",
                "properties": {
                    "item": {"type": "string", "description": "e.g. 'keyboard'"}
                },
                "required": ["item"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "restock_item",
            "description": "Trigger a restock for an item when inventory is low.",
            "parameters": {
                "type": "object",
                "properties": {
                    "item": {"type": "string"},
                    "quantity": {"type": "integer"}
                },
                "required": ["item", "quantity"]
            }
        }
    }
]

### The Chained Reasoning Loop
This is the "Brain". It allows the model to plan a sequence of actions where each tool output feeds the next step.

In [15]:
def run_structured_agent(user_input):
    messages = [{"role": "system", "content": "You are a helpful logistics agent. Use tools to manage inventory."},
                {"role": "user", "content": user_input}]
    
    print(f"USER REQUEST: {user_input}\n")

    # Complex tasks may require multiple steps/tools 
    for i in range(5):  # Safety limit for the loop
        response = client.chat.completions.create(
            model="gpt-4o",
            messages=messages,
            tools=tools
        )
        
        msg = response.choices[0].message
        
        # If the LLM doesn't want to call a tool, we are finished
        if not msg.tool_calls:
            print(f"FINAL RESPONSE: {msg.content}")
            break

        messages.append(msg)

        # Source: Model analyzes request and passes structured arguments 
        for tool_call in msg.tool_calls:
            name = tool_call.function.name
            args = json.loads(tool_call.function.arguments)
            
            print(f"REASONING (Step {i+1}): Decided to call '{name}' with {args}")

            try:
                # Validation and Execution Phase
                if name == "check_stock":
                    valid_args = InventoryCheck(**args)
                    output = backend_system.check_stock(valid_args)
                elif name == "restock_item":
                    valid_args = RestockRequest(**args)
                    output = backend_system.restock_item(valid_args)
                
                print(f"EXECUTION: {output}")

                # Source: Tool output is returned to the system to continue reasoning [cite: 22, 23]
                messages.append({
                    "role": "tool",
                    "tool_call_id": tool_call.id,
                    "name": name,
                    "content": output
                })

            except ValidationError as e:
                print(f"SAFETY BLOCK: Invalid tool arguments generated. {e}")
                return

# Run the Demo
run_structured_agent("Check our keyboard stock. If we have fewer than 5, order enough to reach 20.")

USER REQUEST: Check our keyboard stock. If we have fewer than 5, order enough to reach 20.

REASONING (Step 1): Decided to call 'check_stock' with {'item': 'keyboard'}
EXECUTION: Current stock for keyboard: 0
REASONING (Step 2): Decided to call 'restock_item' with {'item': 'keyboard', 'quantity': 20}
EXECUTION: Successfully ordered 20 units of keyboard.
FINAL RESPONSE: The current stock for keyboards was 0. I have successfully ordered 20 units to replenish the stock.


### Summary of what this code demonstrates:

Decoupled Intelligence: The LLM decides what to do, but the Python backend_system decides how it's done.

* Validation: By using InventoryCheck(**args), we demonstrate that "Safety is enforced at the interface, not the model".

* Agency: Unlike a simple chatbot , this agent actually executes a workflow—checking, calculating the difference (20 - 0), and then restocking.


### Enterprise Audit Log

In [16]:
print("--- ENTERPRISE AUDIT TRAIL ---")
# Audit logs must record every tool call [cite: 51]
for log in backend_system.audit_log:
    print(log)

--- ENTERPRISE AUDIT TRAIL ---
LOG: Checked keyboard. Result: 0
LOG: Restocked keyboard by 20
